In [ ]:
import pygame
import random
import sys
from collections import deque
import math #heurística de risco
import numpy as np # Adicionado para Q-learning

pygame.init()

#configurações do jogo
CELL_SIZE = 80
BRANCO = (255, 255, 255)
CINZA = (200, 200, 200)
PRETO = (0, 0, 0)
VERDE = (0, 200, 0) # Agente / Jogador
VERMELHO = (200, 0, 0) # Perigo
AZUL = (0, 0, 255)
AZUL_CLARO = (100, 100, 255) # Célula inicial fundo
DOURADO = (255, 215, 0) # Ouro
CINZA_ESCURO = (100, 100, 100) # Célula visitada
CINZA_CLARO = (230, 230, 230) # Célula segura inferida (não visitada)

mensagens = []

# ========== CLASSE DO AGENTE INTELIGENTE ORIGINAL (BASEADO EM LÓGICA) ==========
class AgenteInteligente:
    def __init__(self, grid_size):
        self.grid_size = grid_size
        self.posicao = (0, 0)
        self.mapa_conhecido = [[{
            'visitado': False, 'seguro': 'desconhecido',
            'brisa': False, 'fedor': False, 'brilho': False,
            'wumpus': 'desconhecido', 'abismo': 'desconhecido'
            } for _ in range(grid_size)] for _ in range(grid_size)]
        self.mapa_conhecido[0][0]['seguro'] = True
        self.mapa_conhecido[0][0]['visitado'] = True
        self.mapa_conhecido[0][0]['wumpus'] = False
        self.mapa_conhecido[0][0]['abismo'] = False
        self.tem_ouro = False
        self.flecha_disponivel = True
        self.wumpus_morto = False
        self.posicao_wumpus_inferida = None
        self.caminho_planejado = deque()
        self.celulas_a_visitar = set()
        self.atualizar_celulas_a_visitar_adjacentes((0,0))

    def get_vizinhos(self, x, y):
        vizinhos = []
        for dx, dy in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < self.grid_size and 0 <= ny < self.grid_size:
                vizinhos.append((nx, ny))
        return vizinhos

    def atualizar_celulas_a_visitar_adjacentes(self, pos):
        x,y = pos
        for nx, ny in self.get_vizinhos(x, y):
             if 0 <= ny < self.grid_size and 0 <= nx < self.grid_size:
                 celula_vizinha = self.mapa_conhecido[ny][nx]
                 if celula_vizinha['seguro'] == True and not celula_vizinha['visitado']:
                      self.celulas_a_visitar.add((nx,ny))

    def atualizar_percepcoes(self, percepcoes):
        x, y = self.posicao
        if not (0 <= y < self.grid_size and 0 <= x < self.grid_size):
             return
        celula_atual = self.mapa_conhecido[y][x]
        celula_atual['brisa'] = False
        celula_atual['fedor'] = False
        celula_atual['brilho'] = False
        tem_brisa = any("brisa" in p for p in percepcoes)
        tem_fedor = any("fed" in p for p in percepcoes)
        tem_brilho = any("brilho" in p for p in percepcoes)
        celula_atual['brisa'] = tem_brisa
        celula_atual['fedor'] = tem_fedor
        celula_atual['brilho'] = tem_brilho
        vizinhos = self.get_vizinhos(x, y)
        if celula_atual['seguro'] == True and not tem_brisa and not tem_fedor:
            for nx, ny in vizinhos:
                vizinho = self.mapa_conhecido[ny][nx]
                if vizinho['seguro'] == 'desconhecido':
                    vizinho['seguro'] = True
                    vizinho['wumpus'] = False
                    vizinho['abismo'] = False
                    if not vizinho['visitado']:
                        self.celulas_a_visitar.add((nx, ny))
        for nx, ny in vizinhos:
            vizinho = self.mapa_conhecido[ny][nx]
            if vizinho['seguro'] == 'desconhecido':
                if not tem_brisa:
                    vizinho['abismo'] = False
                if not tem_fedor:
                    vizinho['wumpus'] = False
                if vizinho['abismo'] == False and vizinho['wumpus'] == False:
                    vizinho['seguro'] = True
                    if not vizinho['visitado']:
                        self.celulas_a_visitar.add((nx, ny))
        if tem_fedor and not self.wumpus_morto and not self.posicao_wumpus_inferida:
             pos_wumpus_provavel = self.determinar_posicao_wumpus()
             if pos_wumpus_provavel:
                  self.posicao_wumpus_inferida = pos_wumpus_provavel

    def decidir_proxima_acao(self):
        x, y = self.posicao
        if not (0 <= y < self.grid_size and 0 <= x < self.grid_size):
             return None
        celula_atual = self.mapa_conhecido[y][x]
        if celula_atual['brilho'] and not self.tem_ouro:
            self.tem_ouro = True; self.caminho_planejado.clear(); return "pegar_ouro"
        if self.tem_ouro and self.posicao == (0, 0): return "sair"
        if self.tem_ouro:
            if not self.caminho_planejado:
                caminho_volta = self.encontrar_caminho_para_destino_seguro((0, 0))
                if caminho_volta:
                    self.caminho_planejado.extend(caminho_volta[1:])
                else:
                    pass
        if self.caminho_planejado:
            proximo_passo = self.caminho_planejado[0]
            px, py = proximo_passo
            if not (0 <= py < self.grid_size and 0 <= px < self.grid_size):
                self.caminho_planejado.clear()
            else:
                celula_prox = self.mapa_conhecido[py][px]
                eh_ultimo_passo_arriscar = (len(self.caminho_planejado) == 1 and
                                            celula_prox['seguro'] == 'desconhecido' and
                                            not celula_prox['visitado'])
                if celula_prox['seguro'] == True or celula_prox['visitado'] or eh_ultimo_passo_arriscar:
                    passo_a_executar = self.caminho_planejado.popleft()
                    return self.calcular_movimento(passo_a_executar)
                else:
                    self.caminho_planejado.clear()
        if self.posicao_wumpus_inferida and self.flecha_disponivel and not self.wumpus_morto:
            acao_tiro = self.planejar_tiro(self.posicao_wumpus_inferida)
            if acao_tiro:
                return acao_tiro
        self.celulas_a_visitar = {
            pos for pos in self.celulas_a_visitar if 0<=pos[1]<self.grid_size and 0<=pos[0]<self.grid_size and
            self.mapa_conhecido[pos[1]][pos[0]]['seguro'] == True and
            not self.mapa_conhecido[pos[1]][pos[0]]['visitado']
        }
        if self.celulas_a_visitar:
            caminho_para_explorar = self.encontrar_caminho_para_conjunto_seguro(self.celulas_a_visitar)
            if caminho_para_explorar:
                self.caminho_planejado.extend(caminho_para_explorar[1:])
                if self.caminho_planejado:
                    proximo_passo_explorar = self.caminho_planejado.popleft()
                    return self.calcular_movimento(proximo_passo_explorar)
        condicao_vitoria = self.tem_ouro and self.posicao==(0,0)
        if not self.caminho_planejado and not self.celulas_a_visitar and not condicao_vitoria:
            candidatos_risco = self.encontrar_candidatos_risco()
            if candidatos_risco:
                melhor_candidato = self.escolher_melhor_risco(candidatos_risco)
                if melhor_candidato:
                    vizinhos_seguros_do_candidato = set()
                    for v_pos in self.get_vizinhos(melhor_candidato[0], melhor_candidato[1]):
                        vx, vy = v_pos
                        if 0 <= vy < self.grid_size and 0 <= vx < self.grid_size:
                           celula_v = self.mapa_conhecido[vy][vx]
                           if celula_v['seguro'] == True or celula_v['visitado']:
                               vizinhos_seguros_do_candidato.add(v_pos)
                    if vizinhos_seguros_do_candidato:
                        caminho_ate_borda = self.encontrar_caminho_para_conjunto_seguro(vizinhos_seguros_do_candidato)
                        if caminho_ate_borda:
                            self.caminho_planejado.extend(caminho_ate_borda[1:])
                            self.caminho_planejado.append(melhor_candidato)
                            if self.caminho_planejado:
                                return self.calcular_movimento(self.caminho_planejado.popleft())
        if not self.caminho_planejado and self.posicao != (0,0):
            if not self.tem_ouro or (self.tem_ouro and not self.encontrar_caminho_para_destino_seguro((0, 0))):
                 caminho_inicio = self.encontrar_caminho_para_destino_seguro((0, 0))
                 if caminho_inicio and len(caminho_inicio) > 1:
                     self.caminho_planejado.extend(caminho_inicio[1:])
                     if self.caminho_planejado:
                         return self.calcular_movimento(self.caminho_planejado.popleft())
        return None

    def encontrar_candidatos_risco(self):
        candidatos = set()
        for y in range(self.grid_size):
            for x in range(self.grid_size):
                if 0 <= y < self.grid_size and 0 <= x < self.grid_size:
                    celula = self.mapa_conhecido[y][x]
                    if not celula['visitado'] and celula['seguro'] == 'desconhecido':
                        for nx, ny in self.get_vizinhos(x, y):
                             if 0 <= ny < self.grid_size and 0 <= nx < self.grid_size:
                                 if self.mapa_conhecido[ny][nx]['visitado']:
                                     candidatos.add((x,y))
                                     break
        return list(candidatos)

    def escolher_melhor_risco(self, candidatos):
        if not candidatos: return None
        candidatos_avaliados = []
        for pos in candidatos:
            cx, cy = pos
            if not (0 <= cy < self.grid_size and 0 <= cx < self.grid_size): continue
            risco_percebido = 0
            vizinhos_visitados = []
            for nx, ny in self.get_vizinhos(cx, cy):
                 if 0 <= ny < self.grid_size and 0 <= nx < self.grid_size:
                     vizinho = self.mapa_conhecido[ny][nx]
                     if vizinho['visitado']:
                          vizinhos_visitados.append((nx, ny))
                          if vizinho['brisa']: risco_percebido += 1
                          if vizinho['fedor']: risco_percebido += 2
            dist = math.sqrt((self.posicao[0] - cx)**2 + (self.posicao[1] - cy)**2)
            score_final = risco_percebido + dist * 0.5
            candidatos_avaliados.append({'score': score_final, 'risco': risco_percebido, 'dist': dist, 'pos': pos})
        if not candidatos_avaliados: return None
        candidatos_avaliados.sort(key=lambda item: item['score'])
        return candidatos_avaliados[0]['pos']

    def calcular_movimento(self, destino):
        x_atual, y_atual = self.posicao
        x_dest, y_dest = destino
        dx = x_dest - x_atual
        dy = y_dest - y_atual
        if abs(dx) + abs(dy) != 1:
            self.caminho_planejado.clear()
            return None
        if dx == 1: return "mover_leste"
        if dx == -1: return "mover_oeste"
        if dy == 1: return "mover_sul"
        if dy == -1: return "mover_norte"
        return None

    def encontrar_caminho_bfs(self, inicio, condicao_parada, condicao_passagem):
        ix, iy = inicio
        if not (0 <= iy < self.grid_size and 0 <= ix < self.grid_size):
            return None
        if condicao_parada(inicio):
            return [inicio]
        fila = deque([(inicio, [inicio])])
        visitados_bfs = {inicio}
        while fila:
            (x, y), caminho = fila.popleft()
            for nx, ny in self.get_vizinhos(x, y):
                vizinho_pos = (nx, ny)
                if 0 <= ny < self.grid_size and 0 <= nx < self.grid_size:
                    if vizinho_pos not in visitados_bfs and condicao_passagem(vizinho_pos):
                        novo_caminho = caminho + [vizinho_pos]
                        if condicao_parada(vizinho_pos):
                            return novo_caminho
                        visitados_bfs.add(vizinho_pos)
                        fila.append((vizinho_pos, novo_caminho))
        return None

    def encontrar_caminho_para_destino_seguro(self, destino):
        dx, dy = destino
        if not (0 <= dy < self.grid_size and 0 <= dx < self.grid_size):
            return None
        return self.encontrar_caminho_bfs(
            inicio=self.posicao,
            condicao_parada=lambda p: p == destino,
            condicao_passagem=lambda p: 0<=p[1]<self.grid_size and 0<=p[0]<self.grid_size and \
                                       (self.mapa_conhecido[p[1]][p[0]]['seguro'] == True or \
                                        self.mapa_conhecido[p[1]][p[0]]['visitado'])
        )

    def encontrar_caminho_para_conjunto_seguro(self, conjunto_destinos):
        if not conjunto_destinos: return None
        conjunto_valido = {p for p in conjunto_destinos if 0<=p[1]<self.grid_size and 0<=p[0]<self.grid_size}
        if not conjunto_valido:
            return None
        return self.encontrar_caminho_bfs(
            inicio=self.posicao,
            condicao_parada=lambda p: p in conjunto_valido,
            condicao_passagem=lambda p: 0<=p[1]<self.grid_size and 0<=p[0]<self.grid_size and \
                                       (self.mapa_conhecido[p[1]][p[0]]['seguro'] == True or \
                                        self.mapa_conhecido[p[1]][p[0]]['visitado'])
        )

    def determinar_posicao_wumpus(self):
        if self.wumpus_morto: return None
        candidatos_wumpus = set()
        celulas_com_fedor_vizinhas_a_incertos = []
        for y in range(self.grid_size):
            for x in range(self.grid_size):
                if 0 <= y < self.grid_size and 0 <= x < self.grid_size:
                    celula = self.mapa_conhecido[y][x]
                    if celula['visitado'] and celula['fedor']:
                        vizinhos_incertos_desta_celula = set()
                        for nx, ny in self.get_vizinhos(x, y):
                            if 0 <= ny < self.grid_size and 0 <= nx < self.grid_size:
                                vizinho = self.mapa_conhecido[ny][nx]
                                if not vizinho['visitado'] and vizinho['seguro'] != True and vizinho['wumpus'] != False :
                                    vizinhos_incertos_desta_celula.add((nx, ny))
                        if vizinhos_incertos_desta_celula:
                            celulas_com_fedor_vizinhas_a_incertos.append(vizinhos_incertos_desta_celula)
        if not celulas_com_fedor_vizinhas_a_incertos:
            return None
        candidatos_wumpus = celulas_com_fedor_vizinhas_a_incertos[0].copy()
        for i in range(1, len(celulas_com_fedor_vizinhas_a_incertos)):
            candidatos_wumpus.intersection_update(celulas_com_fedor_vizinhas_a_incertos[i])
        if len(candidatos_wumpus) == 1:
            w_pos = candidatos_wumpus.pop()
            wx, wy = w_pos
            if 0 <= wy < self.grid_size and 0 <= wx < self.grid_size:
                if self.mapa_conhecido[wy][wx]['wumpus'] == 'desconhecido':
                     self.mapa_conhecido[wy][wx]['wumpus'] = True
                     if self.mapa_conhecido[wy][wx]['abismo'] != True:
                         self.mapa_conhecido[wy][wx]['seguro'] = False
                return w_pos
            else: return None
        return None

    def planejar_tiro(self, wumpus_pos):
        wx, wy = wumpus_pos
        x, y = self.posicao
        if x == wx:
            if y > wy: return "atirar_norte"
            else: return "atirar_sul"
        elif y == wy:
            if x > wx: return "atirar_oeste"
            else: return "atirar_leste"
        return None

    def atualizar_posicao(self, nova_pos):
        self.posicao = nova_pos
        x, y = self.posicao
        if not (0 <= y < self.grid_size and 0 <= x < self.grid_size):
             return
        celula = self.mapa_conhecido[y][x]
        if not celula['visitado']:
            celula['visitado'] = True
            celula['seguro'] = True
            celula['wumpus'] = False
            celula['abismo'] = False
            if self.posicao in self.celulas_a_visitar:
                self.celulas_a_visitar.remove(self.posicao)

    def wumpus_foi_morto(self, pos_wumpus):
        self.wumpus_morto = True
        self.posicao_wumpus_inferida = None
        wx, wy = pos_wumpus
        if 0 <= wx < self.grid_size and 0 <= wy < self.grid_size:
            celula_wumpus = self.mapa_conhecido[wy][wx]
            celula_wumpus['wumpus'] = False
            if celula_wumpus['abismo'] != True:
                 celula_wumpus['seguro'] = True
                 if not celula_wumpus['visitado']:
                     self.celulas_a_visitar.add((wx,wy))
            for nx, ny in self.get_vizinhos(wx, wy):
                if 0 <= ny < self.grid_size and 0 <= nx < self.grid_size:
                    vizinho = self.mapa_conhecido[ny][nx]
                    vizinho['fedor'] = False
                    self.reavaliar_seguranca_celula(nx, ny)

    def reavaliar_seguranca_celula(self, x, y):
        if not (0 <= y < self.grid_size and 0 <= x < self.grid_size): return
        celula = self.mapa_conhecido[y][x]
        if celula['visitado']:
            celula['seguro'] = True
            return
        if celula['wumpus'] == False and celula['abismo'] == False:
            if celula['seguro'] != True:
                 celula['seguro'] = True
                 if not celula['visitado']:
                      self.celulas_a_visitar.add((x,y))

# ========== NOVA CLASSE DO AGENTE Q-LEARNING (MELHORADA) ==========
class QLearningAgent:
    def __init__(self, grid_size, learning_rate=0.1, discount_factor=0.9, exploration_rate=1.0):
        self.grid_size = grid_size
        self.action_size = 4  # 0: Cima, 1: Baixo, 2: Esquerda, 3: Direita
        self.lr = learning_rate
        self.gamma = discount_factor
        self.epsilon = exploration_rate

        # A Tabela Q agora tem o dobro do tamanho para acomodar o estado "tem_ouro".
        # Parte 1: Estados sem ouro (índices 0 a N-1)
        # Parte 2: Estados com ouro (índices N a 2N-1)
        num_states = grid_size * grid_size
        self.q_table = np.zeros((num_states * 2, self.action_size))

    def _state_to_index(self, state, has_gold):
        """ Converte (x, y, tem_ouro) para um índice único na Tabela Q. """
        base_index = state[1] * self.grid_size + state[0]
        if has_gold:
            # Desloca o índice para a segunda metade da tabela se o agente tem o ouro.
            return base_index + (self.grid_size * self.grid_size)
        return base_index

    def choose_action(self, state, has_gold):
        """ Escolhe uma ação usando a política epsilon-greedy para o estado atual. """
        state_idx = self._state_to_index(state, has_gold)
        if random.uniform(0, 1) < self.epsilon:
            return random.randint(0, self.action_size - 1)  # Explorar
        else:
            return np.argmax(self.q_table[state_idx, :])  # Explotar (usar o conhecimento)

    def learn(self, state, has_gold, action, reward, next_state, next_has_gold, done):
        """ Atualiza a Tabela Q com base na experiência. """
        state_idx = self._state_to_index(state, has_gold)
        next_state_idx = self._state_to_index(next_state, next_has_gold)

        old_value = self.q_table[state_idx, action]

        # O valor futuro é 0 se o episódio terminou.
        next_max = 0 if done else np.max(self.q_table[next_state_idx, :])

        # Fórmula de atualização do Q-learning
        new_value = old_value + self.lr * (reward + self.gamma * next_max - old_value)
        self.q_table[state_idx, action] = new_value


# ========== FUNÇÕES DE INTERFACE E JOGO ==========

def escolher_tamanho():
    # --- Tela de Escolha de Tamanho ---
    tela_escolha = pygame.display.set_mode((500, 400))
    pygame.display.set_caption("Escolha o tamanho da grade")
    fonte_escolha = pygame.font.SysFont(None, 36)
    opcoes_tamanho = [4, 5, 6, 7, 8, 9, 10]
    botoes_tamanho = []
    y_inicial_botao = 50
    espaco_botao = 50
    for i, val in enumerate(opcoes_tamanho):
        rect = pygame.Rect(180, y_inicial_botao + i * espaco_botao, 140, 40)
        botoes_tamanho.append({'rect': rect, 'valor': val})
    while True:
        tela_escolha.fill(BRANCO)
        titulo_surf = fonte_escolha.render("Escolha o tamanho do mapa:", True, PRETO)
        titulo_rect = titulo_surf.get_rect(center=(tela_escolha.get_width() // 2, 25))
        tela_escolha.blit(titulo_surf, titulo_rect)
        mx, my = pygame.mouse.get_pos()
        for botao in botoes_tamanho:
            cor_botao = CINZA
            if botao['rect'].collidepoint(mx, my):
                cor_botao = CINZA_ESCURO
            pygame.draw.rect(tela_escolha, cor_botao, botao['rect'], border_radius=5)
            texto_surf = fonte_escolha.render(f"{botao['valor']} x {botao['valor']}", True, PRETO)
            texto_rect = texto_surf.get_rect(center=botao['rect'].center)
            tela_escolha.blit(texto_surf, texto_rect)
        for evento in pygame.event.get():
            if evento.type == pygame.QUIT:
                pygame.quit(); sys.exit()
            if evento.type == pygame.MOUSEBUTTONDOWN:
                if evento.button == 1:
                    for botao in botoes_tamanho:
                        if botao['rect'].collidepoint(mx, my):
                            global CELL_SIZE
                            tamanho_escolhido = botao['valor']
                            CELL_SIZE = min(90, 800 // tamanho_escolhido)
                            return tamanho_escolhido
        pygame.display.update()

def escolher_modo():
    tela_escolha = pygame.display.set_mode((500, 400))
    pygame.display.set_caption("Escolha o modo")
    fonte_escolha = pygame.font.SysFont(None, 32)

    opcoes_modo = [
        ("Manual", "manual"),
        ("Agente Lógico", "auto"),
        ("Agente Q-learning", "auto_rl")
    ]
    botoes_modo = []
    y_inicial_botao = 80
    espaco_botao = 80
    for i, (nome, valor) in enumerate(opcoes_modo):
        rect = pygame.Rect(100, y_inicial_botao + i * espaco_botao, 300, 60) # Botões mais largos
        botoes_modo.append({'rect': rect, 'valor': valor, 'nome': nome})
    while True:
        tela_escolha.fill(BRANCO)
        titulo_surf = fonte_escolha.render("Como deseja jogar?", True, PRETO)
        titulo_rect = titulo_surf.get_rect(center=(tela_escolha.get_width() // 2, 40))
        tela_escolha.blit(titulo_surf, titulo_rect)
        mx, my = pygame.mouse.get_pos()
        for botao in botoes_modo:
            cor_botao = CINZA
            if botao['rect'].collidepoint(mx, my): cor_botao = CINZA_ESCURO
            pygame.draw.rect(tela_escolha, cor_botao, botao['rect'], border_radius=8)
            texto_surf = fonte_escolha.render(botao['nome'], True, PRETO)
            texto_rect = texto_surf.get_rect(center=botao['rect'].center)
            tela_escolha.blit(texto_surf, texto_rect)
        for evento in pygame.event.get():
            if evento.type == pygame.QUIT: pygame.quit(); sys.exit()
            if evento.type == pygame.MOUSEBUTTONDOWN:
                 if evento.button == 1:
                    for botao in botoes_modo:
                        if botao['rect'].collidepoint(mx, my):
                            return botao['valor']
        pygame.display.update()

def adicionar_mensagem(texto):
    global mensagens
    N_MENSAGENS = 6
    mensagens.append(str(texto))
    if len(mensagens) > N_MENSAGENS:
        mensagens = mensagens[-N_MENSAGENS:]

def mostrar_mensagens(tela, fonte, altura_mapa, largura_mapa):
    y_offset = altura_mapa + 15
    espacamento = fonte.get_height() + 5
    max_linhas = 6
    linhas_mostradas = 0
    for i in range(len(mensagens) -1, -1, -1):
         if linhas_mostradas >= max_linhas: break
         msg = mensagens[i]
         try:
            palavras = msg.split(' ')
            linhas_texto = []
            linha_atual = ''
            for palavra in palavras:
                teste_linha = linha_atual + palavra + ' '
                if fonte.size(teste_linha)[0] < largura_mapa - 20:
                    linha_atual = teste_linha
                else:
                    linhas_texto.append(linha_atual.strip())
                    linha_atual = palavra + ' '
            linhas_texto.append(linha_atual.strip())
            for j in range(len(linhas_texto) -1, -1, -1):
                if linhas_mostradas >= max_linhas: break
                linha_render = linhas_texto[j]
                superficie_texto = fonte.render(linha_render, True, PRETO)
                pos_y_linha = y_offset + (max_linhas - 1 - linhas_mostradas) * espacamento
                tela.blit(superficie_texto, (10, pos_y_linha))
                linhas_mostradas += 1
         except Exception as e:
            print(f"Erro ao renderizar mensagem: '{msg}'. Erro: {e}")

def sorteia_posicoes(grid):
    max_tentativas = 150
    for tentativa in range(max_tentativas):
        posicoes_livres = [(x, y) for y in range(grid) for x in range(grid) if (x, y) != (0, 0)]
        random.shuffle(posicoes_livres)
        if len(posicoes_livres) < 2: return None, None, []
        wumpus = posicoes_livres.pop()
        ouro = posicoes_livres.pop()
        posicoes_para_abismo = [p for p in posicoes_livres if p != (0,1) and p != (1,0)]
        random.shuffle(posicoes_para_abismo)
        quantidade_abismos = max(1, min(len(posicoes_para_abismo), (grid * grid) // 7))
        abismos = []
        for _ in range(quantidade_abismos):
             if posicoes_para_abismo: abismos.append(posicoes_para_abismo.pop())
             elif posicoes_livres: abismos.append(posicoes_livres.pop())
        mapa_teste = [['' for _ in range(grid)] for _ in range(grid)]
        for ax, ay in abismos: mapa_teste[ay][ax] = 'abismo'
        if existe_caminho((0,0), ouro, mapa_teste, grid):
            return wumpus, ouro, abismos
    return None, None, []

def existe_caminho(inicio, fim, mapa_perigo, grid):
     if inicio == fim: return True
     fila = deque([inicio]); visitados = {inicio}
     while fila:
          (x, y) = fila.popleft()
          for dx, dy in [(0,1), (0,-1), (1,0), (-1,0)]:
               nx, ny = x+dx, y+dy
               if 0 <= nx < grid and 0 <= ny < grid and (nx, ny) not in visitados:
                    if mapa_perigo[ny][nx] != 'abismo':
                         if (nx, ny) == fim: return True
                         visitados.add((nx, ny)); fila.append((nx, ny))
     return False

# continua igual, usada pelos agentes Manual e Lógico
def perceber(pos, mapa, grid, modo_jogo='manual'):
    x, y = pos
    percepcoes = []
    prefixo = "Você" if modo_jogo == 'manual' else "Agente"
    if 0 <= y < grid and 0 <= x < grid and mapa[y][x] == "ouro":
        percepcoes.append(f"{prefixo} vê um brilho.")
    for dx, dy in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
        nx, ny = x + dx, y + dy
        if 0 <= nx < grid and 0 <= ny < grid:
            item_vizinho = mapa[ny][nx]
            if item_vizinho == "wumpus":
                if not any("fedor" in p for p in percepcoes):
                    percepcoes.append(f"{prefixo} sente um fedor.")
            elif item_vizinho == "abismo":
                 if not any("brisa" in p for p in percepcoes):
                    percepcoes.append(f"{prefixo} sente uma brisa.")
    return percepcoes

# Função de desenho para o agente lógico (original)
def desenhar_mapa_agente_logico(tela, agente, grid, fonte_principal, fonte_mensagens, pontuacao, altura_mapa, largura_mapa):
    tela.fill(BRANCO)
    fonte_simbolo = pygame.font.SysFont(None, max(18, CELL_SIZE // 4))

    for y in range(grid):
        for x in range(grid):
            rect = pygame.Rect(x * CELL_SIZE, y * CELL_SIZE, CELL_SIZE, CELL_SIZE)
            celula_agente = agente.mapa_conhecido[y][x]

            cor_fundo = PRETO
            if celula_agente['visitado']: cor_fundo = CINZA_ESCURO
            elif celula_agente['seguro'] == True: cor_fundo = CINZA_CLARO
            elif celula_agente['seguro'] == False: cor_fundo = VERMELHO
            if (x,y) == (0,0): cor_fundo = AZUL_CLARO

            pygame.draw.rect(tela, cor_fundo, rect)
            pygame.draw.rect(tela, CINZA, rect, 1)

            simbolos = []
            if celula_agente['visitado']:
                 if celula_agente['brisa']: simbolos.append('B')
                 if celula_agente['fedor']: simbolos.append('F')
                 if celula_agente['brilho']: simbolos.append('G')
            if celula_agente['wumpus'] == True and not agente.wumpus_morto: simbolos.append('W?')
            if celula_agente['abismo'] == True: simbolos.append('A?')
            if celula_agente['seguro'] == True and not celula_agente['visitado']: simbolos.append('S?')

            if simbolos:
                texto_simbolos = fonte_simbolo.render(" ".join(simbolos), True, BRANCO if cor_fundo in [PRETO, VERMELHO, CINZA_ESCURO] else PRETO)
                texto_rect = texto_simbolos.get_rect(topleft=(rect.x + 4, rect.y + 4))
                tela.blit(texto_simbolos, texto_rect)

            if (x, y) == agente.posicao:
                pygame.draw.circle(tela, VERDE, rect.center, CELL_SIZE // 3)

    pontuacao_txt = fonte_principal.render(f"Pontos: {pontuacao}", True, PRETO)
    tela.blit(pontuacao_txt, (10, altura_mapa + 5))
    mostrar_mensagens(tela, fonte_mensagens, altura_mapa, largura_mapa)

def desenhar_mapa_completo_rl(tela, pos_agente, mapa_real, grid, fonte_principal, fonte_mensagens, pontuacao, altura_mapa, largura_mapa, wumpus_morto, tem_ouro_agente):
    tela.fill(BRANCO)
    fonte_simbolo = pygame.font.SysFont(None, max(24, CELL_SIZE // 3))
    for y in range(grid):
        for x in range(grid):
            rect = pygame.Rect(x * CELL_SIZE, y * CELL_SIZE, CELL_SIZE, CELL_SIZE)
            cor_fundo = CINZA_CLARO
            if (x,y) == (0,0): cor_fundo = AZUL_CLARO
            pygame.draw.rect(tela, cor_fundo, rect)

            conteudo = mapa_real[y][x]
            if conteudo == 'wumpus' and not wumpus_morto:
                wumpus_txt = fonte_simbolo.render("W", True, VERMELHO)
                tela.blit(wumpus_txt, wumpus_txt.get_rect(center=rect.center))
            elif conteudo == 'abismo':
                pygame.draw.circle(tela, PRETO, rect.center, CELL_SIZE // 3)
            elif conteudo == 'ouro':
                 ouro_txt = fonte_simbolo.render("O", True, DOURADO)
                 tela.blit(ouro_txt, ouro_txt.get_rect(center=rect.center))

            pygame.draw.rect(tela, CINZA, rect, 1)

    agent_rect = pygame.Rect(pos_agente[0] * CELL_SIZE, pos_agente[1] * CELL_SIZE, CELL_SIZE, CELL_SIZE)
    cor_agente = DOURADO if tem_ouro_agente else VERDE
    pygame.draw.circle(tela, cor_agente, agent_rect.center, CELL_SIZE // 3)

    pontuacao_txt = fonte_principal.render(f"Pontos: {pontuacao}", True, PRETO)
    tela.blit(pontuacao_txt, (10, altura_mapa + 5))
    mostrar_mensagens(tela, fonte_mensagens, altura_mapa, largura_mapa)


def desenhar_mapa_manual(tela, jogador_pos, visitados_manual, grid, fonte_principal, fonte_mensagens, pontuacao, altura_mapa, largura_mapa, tem_flecha):
    tela.fill(BRANCO)
    for y in range(grid):
        for x in range(grid):
            rect = pygame.Rect(x * CELL_SIZE, y * CELL_SIZE, CELL_SIZE, CELL_SIZE)
            pos_atual = (x,y)
            cor_fundo = PRETO
            if pos_atual in visitados_manual: cor_fundo = CINZA_ESCURO
            if pos_atual == (0,0): cor_fundo = AZUL_CLARO
            pygame.draw.rect(tela, cor_fundo, rect)
            pygame.draw.rect(tela, CINZA, rect, 1)
            if pos_atual == jogador_pos:
                pygame.draw.circle(tela, VERDE, rect.center, CELL_SIZE // 3)
    pontuacao_txt = fonte_principal.render(f"Pontos: {pontuacao} | Flecha: {'Sim' if tem_flecha else 'Não'}", True, PRETO)
    tela.blit(pontuacao_txt, (10, altura_mapa + 5))
    mostrar_mensagens(tela, fonte_mensagens, altura_mapa, largura_mapa)

def mostrar_tela_final(tela, ganhou, pontuacao, fonte):
    largura = tela.get_width(); altura = tela.get_height()
    texto = "Vitória!" if ganhou else "Game Over!"
    texto_pontos = f"Pontuação Final: {pontuacao}"
    texto2 = "Clique ou tecle algo para jogar novamente"
    cor_titulo = VERDE if ganhou else VERMELHO
    overlay = pygame.Surface((largura, altura), pygame.SRCALPHA)
    overlay.fill((200, 200, 200, 210))
    tela.blit(overlay, (0,0))
    titulo_surf = fonte.render(texto, True, cor_titulo)
    pontos_surf = fonte.render(texto_pontos, True, PRETO)
    subtitulo_surf = fonte.render(texto2, True, PRETO)
    titulo_rect = titulo_surf.get_rect(center=(largura // 2, altura // 2 - 60))
    pontos_rect = pontos_surf.get_rect(center=(largura // 2, altura // 2 - 10))
    subtitulo_rect = subtitulo_surf.get_rect(center=(largura // 2, altura // 2 + 40))
    tela.blit(titulo_surf, titulo_rect)
    tela.blit(pontos_surf, pontos_rect)
    tela.blit(subtitulo_surf, subtitulo_rect)
    pygame.display.update()
    esperando = True
    while esperando:
        for evento in pygame.event.get():
            if evento.type == pygame.QUIT: pygame.quit(); sys.exit()
            if evento.type == pygame.MOUSEBUTTONDOWN or evento.type == pygame.KEYDOWN :
                esperando = False
        pygame.time.Clock().tick(15)

def iniciar_jogo():
    global mensagens, CELL_SIZE

    GRID_SIZE = escolher_tamanho()
    modo = escolher_modo()

    ALTURA_MAPA = GRID_SIZE * CELL_SIZE
    LARGURA_MAPA = GRID_SIZE * CELL_SIZE
    ALTURA_MSG = 180
    tela = pygame.display.set_mode((LARGURA_MAPA, ALTURA_MAPA + ALTURA_MSG))
    pygame.display.set_caption("Mundo de Wumpus")
    fonte_principal = pygame.font.SysFont(None, max(24, CELL_SIZE // 3))
    fonte_mensagens = pygame.font.SysFont(None, max(20, CELL_SIZE // 4))

    mensagens = []
    mapa_real = [["" for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]

    pos_wumpus, pos_ouro, pos_abismos = sorteia_posicoes(GRID_SIZE)
    if pos_wumpus is None:
        print("Falha ao gerar mapa válido. Encerrando."); pygame.quit(); sys.exit()

    mapa_real[pos_wumpus[1]][pos_wumpus[0]] = "wumpus"
    mapa_real[pos_ouro[1]][pos_ouro[0]] = "ouro"
    for ax, ay in pos_abismos: mapa_real[ay][ax] = "abismo"

    # Variáveis de estado do jogo
    pos_agente = (0, 0)
    pontuacao = 0
    tem_ouro = False
    wumpus_morto = False
    vivo = True
    vitoria = False
    flecha_disponivel = True
    visitados_manual = set([(0,0)])

    adicionar_mensagem("Bem-vindo ao Mundo de Wumpus!")
    if modo == 'manual':
         adicionar_mensagem("Setas: Mover | W/A/S/D: Mirar | Espaço: Atirar")
    elif modo == 'auto':
         adicionar_mensagem("Agente Lógico iniciado.")
    elif modo == 'auto_rl':
        adicionar_mensagem("Agente Q-learning iniciado.")

    # ==========================================================
    # LÓGICA DE INICIALIZAÇÃO E TREINAMENTO DOS AGENTES
    # ==========================================================
    agente_logico = None
    rl_agent = None

    if modo == 'auto_rl':
        # --- PARÂMETROS DE TREINO ---
        EPISODES = 20000
        MAX_STEPS_PER_EPISODE = GRID_SIZE * GRID_SIZE * 3
        INITIAL_EPSILON = 1.0
        FINAL_EPSILON = 0.01
        epsilon_decay = (INITIAL_EPSILON - FINAL_EPSILON) / (EPISODES * 0.8)

        # --- CONFIGURAÇÃO DA VISUALIZAÇÃO DO TREINO ---
        VISUALIZAR_TREINO = True # Mudar para False para treino ultra-rápido
        VISUALIZAR_A_CADA_N_EPISODIOS = 250
        DELAY_VISUALIZACAO = 50 # ms entre frames na visualização

        rl_agent = QLearningAgent(GRID_SIZE, exploration_rate=INITIAL_EPSILON)
        adicionar_mensagem(f"Treinando por {EPISODES} episódios...")
        desenhar_mapa_completo_rl(tela, pos_agente, mapa_real, GRID_SIZE, fonte_principal, fonte_mensagens, 0, ALTURA_MAPA, LARGURA_MAPA, False, False)
        pygame.display.update()
        pygame.time.delay(500)

        for episode in range(EPISODES):
            state = (0, 0)
            has_gold_training = False

            # Decide se este episódio será visualizado
            visualize_this_episode = VISUALIZAR_TREINO and (episode % VISUALIZAR_A_CADA_N_EPISODIOS == 0)

            if visualize_this_episode:
                mensagens.clear()
                adicionar_mensagem(f"Visualizando treino: Episódio {episode}")

            for step in range(MAX_STEPS_PER_EPISODE):
                # Para a janela não travar durante treinos longos
                if episode % 100 == 0:
                    pygame.event.get()

                action = rl_agent.choose_action(state, has_gold_training)

                nx, ny = state
                if action == 0: ny -= 1
                elif action == 1: ny += 1
                elif action == 2: nx -= 1
                elif action == 3: nx += 1
                nx = max(0, min(GRID_SIZE - 1, nx)); ny = max(0, min(GRID_SIZE - 1, ny))
                next_state = (nx, ny)

                reward = -1; done = False; next_has_gold = has_gold_training
                conteudo = mapa_real[next_state[1]][next_state[0]]

                if not has_gold_training:
                    if conteudo == 'ouro':
                        reward += 1000; next_has_gold = True
                    elif conteudo in ['wumpus', 'abismo']:
                        reward -= 1000; done = True
                else:
                    if next_state == (0, 0):
                        reward += 1000; done = True
                    elif conteudo in ['wumpus', 'abismo']:
                        reward -= 1000; done = True

                rl_agent.learn(state, has_gold_training, action, reward, next_state, next_has_gold, done)
                state = next_state
                has_gold_training = next_has_gold

                if visualize_this_episode:
                    desenhar_mapa_completo_rl(tela, state, mapa_real, GRID_SIZE, fonte_principal, fonte_mensagens, 0, ALTURA_MAPA, LARGURA_MAPA, False, has_gold_training)
                    pygame.display.update()
                    pygame.time.delay(DELAY_VISUALIZACAO)

                if done: break

            if rl_agent.epsilon > FINAL_EPSILON:
                rl_agent.epsilon -= epsilon_decay

        adicionar_mensagem("Treinamento concluído. Iniciando execução...")
        desenhar_mapa_completo_rl(tela, (0,0), mapa_real, GRID_SIZE, fonte_principal, fonte_mensagens, 0, ALTURA_MAPA, LARGURA_MAPA, False, False)
        pygame.display.update()
        pygame.time.delay(1000)
        rl_agent.epsilon = 0

    elif modo == 'auto':
        agente_logico = AgenteInteligente(GRID_SIZE)
        percepcoes_atuais = perceber(pos_agente, mapa_real, GRID_SIZE, modo)
        agente_logico.atualizar_percepcoes(percepcoes_atuais)
        for p in percepcoes_atuais: adicionar_mensagem(p)
        desenhar_mapa_agente_logico(tela, agente_logico, GRID_SIZE, fonte_principal, fonte_mensagens, pontuacao, ALTURA_MAPA, LARGURA_MAPA)
        pygame.display.update()
        pygame.time.delay(1000)
    else: # MODO MANUAL
        percepcoes_atuais = perceber(pos_agente, mapa_real, GRID_SIZE, modo)
        for p in percepcoes_atuais: adicionar_mensagem(p)
        desenhar_mapa_manual(tela, pos_agente, visitados_manual, GRID_SIZE, fonte_principal, fonte_mensagens, pontuacao, ALTURA_MAPA, LARGURA_MAPA, flecha_disponivel)
        pygame.display.update()

    # --- Loop Principal do Jogo ---
    rodando = True
    clock = pygame.time.Clock()
    tempo_agente = 0
    DELAY_AGENTE = 300 # Aumentado um pouco para melhor visualização

    while rodando:
        dt = clock.tick(30)

        # --- Lógica de Eventos (para fechar janela e modo manual) ---
        for evento in pygame.event.get():
            if evento.type == pygame.QUIT:
                rodando = False; continue

            if modo == "manual" and vivo and not vitoria:
                acao_realizada_manual = False; nova_pos_manual = pos_agente; acao_tiro_manual = False
                if evento.type == pygame.KEYDOWN:
                    x, y = pos_agente
                    if evento.key == pygame.K_UP: nova_pos_manual = (x, y - 1)
                    elif evento.key == pygame.K_DOWN: nova_pos_manual = (x, y + 1)
                    elif evento.key == pygame.K_LEFT: nova_pos_manual = (x - 1, y)
                    elif evento.key == pygame.K_RIGHT: nova_pos_manual = (x + 1, y)

                    if nova_pos_manual != pos_agente:
                        if 0 <= nova_pos_manual[0] < GRID_SIZE and 0 <= nova_pos_manual[1] < GRID_SIZE:
                            pos_agente = nova_pos_manual; visitados_manual.add(pos_agente)
                            pontuacao -= 1; mensagens.clear(); acao_realizada_manual = True
                            adicionar_mensagem(f"Moveu para {pos_agente}.")
                            conteudo = mapa_real[pos_agente[1]][pos_agente[0]]
                            if conteudo == "abismo": adicionar_mensagem("CAIU NO ABISMO!"); pontuacao -= 1000; vivo = False
                            elif conteudo == "wumpus" and not wumpus_morto: adicionar_mensagem("DEVORADO!"); pontuacao -= 1000; vivo = False
                            elif conteudo == "ouro":
                                if not tem_ouro: tem_ouro = True; mapa_real[pos_agente[1]][pos_agente[0]] = ""; adicionar_mensagem("PEGOU O OURO!"); pontuacao += 1000
                            if vivo:
                                percepcoes_atuais = perceber(pos_agente, mapa_real, GRID_SIZE, modo)
                                if not percepcoes_atuais and conteudo == "": adicionar_mensagem("Seguro.")
                                for p in percepcoes_atuais: adicionar_mensagem(p)
                if vivo and tem_ouro and pos_agente == (0, 0):
                    adicionar_mensagem("Você voltou com o ouro! Vitória!"); vitoria = True
                if acao_realizada_manual or not vivo or vitoria:
                    desenhar_mapa_manual(tela, pos_agente, visitados_manual, GRID_SIZE, fonte_principal, fonte_mensagens, pontuacao, ALTURA_MAPA, LARGURA_MAPA, flecha_disponivel)
                    pygame.display.update()

        # --- Lógica de atualização dos Agentes Automáticos ---
        if modo in ["auto", "auto_rl"] and vivo and not vitoria:
            tempo_agente += dt
            if tempo_agente >= DELAY_AGENTE:
                tempo_agente = 0

                # --- AGENTE LÓGICO  ---
                if modo == "auto":
                    mensagens.clear()
                    acao = agente_logico.decidir_proxima_acao()
                    acao_realizada_agente = False

                    if acao is None:
                        adicionar_mensagem("Agente Lógico: Não sabe o que fazer. Desistindo.")
                        vivo = False; acao_realizada_agente = True
                    elif acao.startswith("mover_"):
                        x, y = pos_agente; nx, ny = x, y
                        if acao == "mover_norte": ny -= 1
                        elif acao == "mover_sul": ny += 1
                        elif acao == "mover_leste": nx += 1
                        elif acao == "mover_oeste": nx -= 1
                        pos_agente = (nx,ny); pontuacao -= 1
                        agente_logico.atualizar_posicao(pos_agente)
                        adicionar_mensagem(f"Agente Lógico move para {pos_agente}.")
                        acao_realizada_agente = True
                    elif acao == "pegar_ouro":
                        tem_ouro = True; mapa_real[pos_agente[1]][pos_agente[0]] = ""; agente_logico.tem_ouro = True
                        adicionar_mensagem("Agente Lógico pegou o ouro."); pontuacao += 1000
                        acao_realizada_agente = True
                    elif acao == "sair":
                        if pos_agente == (0,0) and tem_ouro:
                             adicionar_mensagem("Agente Lógico voltou com o ouro! Vitória!")
                             vitoria = True
                        acao_realizada_agente = True

                    if vivo and acao_realizada_agente:
                        conteudo = mapa_real[pos_agente[1]][pos_agente[0]]
                        if conteudo == "abismo" or (conteudo == "wumpus" and not wumpus_morto):
                            adicionar_mensagem("Agente Lógico morreu!")
                            pontuacao -= 1000
                            vivo = False
                        else:
                           percepcoes_atuais = perceber(pos_agente, mapa_real, GRID_SIZE, modo)
                           agente_logico.atualizar_percepcoes(percepcoes_atuais)
                           for p in percepcoes_atuais: adicionar_mensagem(p)

                    desenhar_mapa_agente_logico(tela, agente_logico, GRID_SIZE, fonte_principal, fonte_mensagens, pontuacao, ALTURA_MAPA, LARGURA_MAPA)
                    pygame.display.update()

                # --- AGENTE Q-LEARNING ---
                elif modo == "auto_rl":
                    mensagens.clear()
                    action = rl_agent.choose_action(pos_agente, tem_ouro)
                    action_map = {0: 'Cima', 1: 'Baixo', 2: 'Esquerda', 3: 'Direita'}
                    adicionar_mensagem(f"Agente Q-Learning move para {action_map[action]}.")

                    nx, ny = pos_agente
                    if action == 0: ny -= 1
                    elif action == 1: ny += 1
                    elif action == 2: nx -= 1
                    elif action == 3: nx += 1
                    pos_agente = (nx, ny)
                    pontuacao -= 1

                    conteudo = mapa_real[ny][nx]
                    if conteudo == 'abismo' or (conteudo == 'wumpus' and not wumpus_morto):
                        adicionar_mensagem("Agente morreu!"); pontuacao -= 1000; vivo = False
                    elif conteudo == 'ouro':
                        adicionar_mensagem("Agente encontrou o ouro!"); pontuacao += 1000
                        tem_ouro = True
                        mapa_real[ny][nx] = ""

                    if vivo and tem_ouro and pos_agente == (0,0):
                        adicionar_mensagem("Agente voltou com o ouro! Vitória!"); vitoria = True

                    desenhar_mapa_completo_rl(tela, pos_agente, mapa_real, GRID_SIZE, fonte_principal, fonte_mensagens, pontuacao, ALTURA_MAPA, LARGURA_MAPA, wumpus_morto, tem_ouro)
                    pygame.display.update()

        if not vivo or vitoria:
            rodando = False

    pygame.time.delay(1000)
    mostrar_tela_final(tela, vitoria, pontuacao, fonte_principal)

if __name__ == "__main__":
    try:
        while True:
             iniciar_jogo()
    except SystemExit:
         pass
    except Exception as e:
        print(f"\nERRO INESPERADO: {e}")
        import traceback
        traceback.print_exc()
    finally:
        pygame.quit()
